## Ngày 7: Đánh giá Hệ thống Đề xuất
Các chỉ số thường dùng: MAE, RMSE, Precision@k, Recall@k

### Mục đích của việc đánh giá hệ thống
- Xác định độ chính xác và hiệu quả của hệ thống, từ đó cải thiện các thuật toán gợi ý
- Phân loại đánh giá:
    + Offline Evailuation: Dùng các dữ liệu lịch sử và các chỉ số đo lường như MAE, RMSE, Precission@K, Recall@K, F1-score, NDCG...
    + Online Evaluation: Sử dụng A/B testing hoặc các phương pháp trực tuyến để đo lường sự hài lòng của người dùng khi sử dụng hệ thống

### Các chỉ số đánh giá quan trọng
- Đối với bài toán Rating Prediction (Dự đoán điểm số):
    + MAE: Trung bình sai số tuyệt đối giữa giá trị dự đoán và giá trị thực
    + RMSE: Căn bậc hai của trung bình phương sai số, nhấn mạnh các sai lệnh lớn hơn
- Đối với bài taons Top-K Recommendation (Gợi ý sản phẩm):
    + Precission@K: Tỉ lệ các sản phẩm gợi ý đúng trên tổng số sản phẩm gợi ý
    + Recall@K: Tỉ lệ các sản phẩm gợi ý đúng trên tổng số sản phẩm người dùng thực sự thích
    + F1-score: Trung bình điều hòa của Precission và Recall
    + NDCG: Đo lường mức độ hiệu quả của danh sách gợi ý có thứ tự (ranking) dựa trên vị trí của các sản phẩm đúng

### Ví dụ thực hành với Code: Đánh giá RMSE
Giả sử ta có giá trị rating y_true và các giá trị dự đoán y_pred, ta có thể tính RMSE như sau:

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

# Vi du duw lieu thuc te va du doan
y_true = [3, 4, 5, 2, 1]
y_pred = [2.5, 4, 5, 3, 1.5]

# Tinh MSE
mse = mean_squared_error(y_true, y_pred)
# Tinh RMSE
rmse = sqrt(mse)
# => RMSE càng nhỏ cho thấy mô hình dự đoán càng chính xác

- Chọn đúng chỉ số: Tùy theo bài toán và mục tiêu của hệ thống, nên chọn chỉ số phù hợp để đánh giá. Ví dụ nếu quan tâm đến việc xếp hạng, NDCG và Precission@K có thể phù hợp hơn.
- Offline và Online: Đừng quên kết hợp đánh giá offline và online để có cái nhìn toàn diện. Offline giúp đánh giá mô hình trên dữ liệu lịch sử, trong khi online phản ánh trải nghiệm thực tế của người dùng
- Theo doi theo thời gian: Hệ thống đề xuất có thể thay đổi theo thời gian (model drift), nên việc theo dõi và cập nhật các chỉ số định kỳ là cần thiết
    

### Thực hành trên bộ dữ liệu MovieLens

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squered_error
from sklearn.metrics.pairwise import consine_similarity
from math import sqrt

# Load data
ratings = pd.read_csv("../Datasets/MovieLens/ratings.csv")
ratings.head()

In [ ]:
# Preprcessing data
# Xóa các bản ghi trùng lặp
ratings.drop_duplicates(inplace=True)

# Chuyển đổi timestamp từ dạng số sang datetime (nếu cần)
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.infor()

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)
print(train_data.shape[0])
print(test_data.shape[0])

In [ ]:
# Tạo ma trận người dùng - phim từ dữ liệu huấn luyện
train_matrix = train_data.pivot(index='userId', columns='movieId', values='rating')
# Điền giá trị missing bằng 0
train_matrix.fillna(0, inplace=True)
print(train_matrix.head())

In [ ]:
# Tính toán cóine similarity giữa các người dùng
user_similarity = consine_similarity(train_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=train_matrix.index, columns=train_matrix.index)
user_similarity_df.head()

In [ ]:
# Hàm dự đoán raing cho một cặp user và movie dựa trên user-user collaborative filtering
def predict_rating(user_id, movie_id, train_matrix, user_similarity_df):
    if movie_id not in train_matrix.columns:
        # Không thể dự đoán nếu movie không có trong dữ liệu huấn luyện
        return np.nan
    # Lấy độ tương đồng của user_id trên ma trận huấn luyện
    sim_scores = user_similarity_df[user_id]
    # Lấy cột rating của movie_id trên ma trận huấn luyện
    movie_ratings = train_matrix[movie_id]
    # Chỉ xét những ngưởi dùng đã đánh giá movie (rating > 0)
    mask = movie_ratings > 0
    if mask.sum() == 0:
        # Không có ai đánh giá movie này
        return np.nan    
    # Dự đoán rating theo công thức trọng số: tổng (similarity * rating) chia cho tổng similarity
    numerator = (sim_scores[mask] * movie_ratings[mask]).sum()
    denominator = np.abs(sim_scores[mask]).sum()
    if denominator == 0:
        return np.nan
    return numerator/denominator

In [ ]:
# Dự đoán rating cho mỗi cặp trong tập kiểm tra và tính RMSE
predictions = []
actuals = []

for index, row in test_data.iterrows():
    user = row['userId']
    movie = row['movieId']
    actual_rating = row['rating']
    pred = predict_rating(user, movie, train_matrix, user_similarity_df)
    # Nếu không thể dự đoán được, sử dụng trung bình rating của dữ liệu huấn luyện
    if np.isnan(pred):
        pred = train_data['rating'].mean()
    predictions.append(pred)
    actuals.append(actual_rating)

rmse = sqrt(mean_squared_error(actuals, predictions))
print(rmse)